In [129]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cvxopt
import yfinance as yf
import pypfopt
from pypfopt import risk_models
from pypfopt import plotting
from pypfopt import expected_returns
from pypfopt import EfficientFrontier
from pypfopt import DiscreteAllocation
from pypfopt import objective_functions
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
import datetime
import talib

In [130]:
# TWS API implementation
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.common import TickerId
from threading import Event
from ibapi.contract import Contract
from ibapi.order import Order

class PositionHandler(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.positions = {}
        self.finished = Event()

    def position(self, account: str, contract, pos: float, avgCost: float):
        self.positions[contract.symbol] = (pos, avgCost)

    def positionEnd(self):
        print("End of positions")
        self.finished.set()

# Create the client and connect to TWS
client = PositionHandler()
client.connect("localhost", 7497, clientId=0)

def get_stock_value(symbol):
    client = PositionHandler()
    client.connect("localhost", 7497, clientId=0)

    # Request positions
    client.reqPositions()

    # Wait until we have all positions
    client.finished.wait()

    # Now we can disconnect
    client.disconnect()

    # Get the position and average cost of the specified symbol
    pos, avgCost = client.positions.get(symbol, (0, 0))

    return pos * avgCost

def make_contract(symbol, sec_type="STK", exch="SMART", prim_exch="ISLAND", curr="USD"):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = sec_type
    contract.exchange = exch
    contract.primaryExchange = prim_exch
    contract.currency = curr
    return contract

def make_order(action, quantity, price = None):
    order = Order()
    order.action = action
    order.totalQuantity = quantity
    order.orderType = "MKT" if price is None else "LMT"
    order.lmtPrice = price
    return order

ERROR -1 502 Couldn't connect to TWS. Confirm that "Enable ActiveX and Socket EClients" 
is enabled and connection port is the same as "Socket Port" on the 
TWS "Edit->Global Configuration...->API->Settings" menu. Live Trading ports: 
TWS: 7496; IB Gateway: 4001. Simulated Trading ports for new installations 
of version 954.1 or newer:  TWS: 7497; IB Gateway: 4002


In [131]:
def get_current_price(stock):
    ticker = yf.Ticker(stock)
    return ticker.info['currentPrice']

def getStockPrice(s, lastYearMore = False):
    end_date = pd.Timestamp.now()
    start_date = end_date - pd.DateOffset(days=42)
    raw = yf.download(s, start=start_date, end=end_date)
    p = raw["Adj Close"].dropna(how="all")
    # Add cash into the dataset

    # Copy last year data to achieve 1.5x weight
    # Add weights
    if lastYearMore:
        recent_year = end_date - pd.DateOffset(years=1)
        recent_data = p[p.index >= recent_year]
        weighted_prices = pd.concat([p, recent_data, recent_data * 0.5])
        return weighted_prices.dropna()
    return p.dropna()

def predict_trading(prices, shares, model="RF"):
    for s in prices.columns:
        bb_upper, bb_middle, bb_lower = talib.BBANDS(prices[s], timeperiod=20)
        bb_percentage = (prices[s] - bb_lower) / (bb_upper - bb_lower)
        mean = bb_percentage.mean()
        bb_percentage = bb_percentage.fillna(mean)
        X = bb_percentage.values[:-1].reshape(-1,1)
        y = np.where(prices[s].values[1:] > bb_upper[:-1], -1, 1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        if model == 'RF':
            estimator  = RandomForestClassifier(max_depth=2, random_state=0)
        elif model == 'SVM':
            estimator = SVR(kernel='rbf')
        elif model == 'DT':
            estimator = DecisionTreeClassifier(max_depth=2, random_state=0)
        else:
            raise ValueError(f"Unknown model: {model}")

        estimator.fit(X_train, y_train)
        y_pred = estimator.predict(X_test)

        # For each stock, fetch its current hold stock value as capital by TWS API
        # capital = get_stock_value(s)
        # If cannot use TWS API, comment above and use below
        capital = shares[s] * (get_current_price(s))
        position = 0

        for i in range(len(y_pred)):
            if y_pred[i] == 1 and position != 1:  # Buy signal
                position = 1
                # Fetch current price
                current_price = get_current_price(s)
                num_shares_to_buy = (0.05 * capital) // current_price
                print(f"Buy {num_shares_to_buy} shares of {s}")
                shares[s] += num_shares_to_buy
                # Use TWS API to do position
                # COMMENT BELOW IF JUST WANT TO SEE PERFORMANCE
                #contract = make_contract(s)
                #order = make_order("BUY", num_shares_to_buy)
                #client.placeOrder(client.nextOrderId(), contract, order)

            elif y_pred[i] == -1 and position != -1:  # Sell signal
                position = -1
                current_price = get_current_price(s)
                num_shares_to_sell = (0.05 * capital) // current_price
                print(f"Sell {num_shares_to_sell} shares of {s}")
                shares[s] -= num_shares_to_sell
                # Use TWS API to do position
                # COMMENT BELOW IF JUST WANT TO SEE PERFORMANCE
                #contract = make_contract(s)
                #order = make_order("SELL", num_shares_to_sell)
                #client.placeOrder(client.nextOrderId(), contract, order)

            elif y_pred[i] == 0 and position != 0:  # Exit position
                position = 0
                print("No change " + s)

        final_capital = capital
        print("Final Capital: ", final_capital)
    return shares

In [132]:
stock_shares = {
    'AAPL': 268,
    'AMZN': 389,
    'BSX': 939,
    'CI': 183,
    'COST': 95,
    'CVX': 577,
    'GOOGL': 425,
    'GS': 258,
    'HSBC': 1309,
    'LMT': 154,
    'MCD': 185,
    'MRK': 449,
    'MSFT': 177,
    'PFE': 1931,
    'SEDG': 138,
    'TSLA': 203,
    'NVDA': 138
}
stocks = ['PFE', 'COST', 'AMZN', 'GOOGL', 'MRK', 'TSLA', 'LMT', 'BSX', 'HSBC', 'MSFT', 'SEDG', 'CVX', 'MCD', 'AAPL', 'GS', 'CI', 'NVDA']
stock_prices = getStockPrice(stocks, lastYearMore=False)
index = ['^SPX']
index_price = getStockPrice(index)

[*********************100%***********************]  17 of 17 completed
[*********************100%***********************]  1 of 1 completed


In [133]:
stock_shares = predict_trading(stock_prices, stock_shares)

Buy 13.0 shares of AAPL
Final Capital:  51983.96
Buy 19.0 shares of AMZN
Final Capital:  50710.04000000001
Buy 46.0 shares of BSX
Final Capital:  50790.51
Buy 9.0 shares of CI
Final Capital:  51349.8
Buy 4.0 shares of COST
Final Capital:  51146.1
Buy 28.0 shares of CVX
Final Capital:  90790.95
Buy 21.0 shares of GOOGL
Final Capital:  50872.5
Buy 12.0 shares of GS
Final Capital:  83215.32
Buy 65.0 shares of HSBC
Final Capital:  51862.579999999994
Buy 7.0 shares of LMT
Final Capital:  70898.52
Buy 9.0 shares of MCD
Sell 9.0 shares of MCD
Buy 9.0 shares of MCD
Final Capital:  55205.850000000006
Buy 22.0 shares of MRK
Final Capital:  51810.11
Buy 8.0 shares of MSFT
Final Capital:  60275.58
Buy 6.0 shares of NVDA
Final Capital:  58376.759999999995
Buy 96.0 shares of PFE
Final Capital:  70829.08
Buy 6.0 shares of SEDG
Final Capital:  37128.9
Buy 10.0 shares of TSLA
Final Capital:  53139.31


In [134]:
stock_shares

{'AAPL': 281.0,
 'AMZN': 408.0,
 'BSX': 985.0,
 'CI': 192.0,
 'COST': 99.0,
 'CVX': 605.0,
 'GOOGL': 446.0,
 'GS': 270.0,
 'HSBC': 1374.0,
 'LMT': 161.0,
 'MCD': 194.0,
 'MRK': 471.0,
 'MSFT': 185.0,
 'PFE': 2027.0,
 'SEDG': 144.0,
 'TSLA': 213.0,
 'NVDA': 144.0}